## 단어 자동 완성

> 4개의 글자를 가진 단어를 학습시켜, 3글자만 주어지면 나머지 한 글자를 추천하여 단어를 완성하는 프로그램

In [3]:
import tensorflow as tf
import numpy as np

char_arr = ['a', 'b', 'c', 'd', 'e', 'f', 'g',
            'h', 'i', 'j', 'k', 'l', 'm', 'n',
            'o', 'p', 'q', 'r', 's', 't', 'u',
            'v', 'w', 'x', 'y', 'z']

# one-hot 인코딩 사용 및 디코딩을 위한 배열을 만듦
# {'a': 0, 'b': 1, 'c': 2, ..., 'j': 9, 'k', 10, ...}
num_dic = {n: i for i, n in enumerate(char_arr)}
dic_len = len(num_dic)

# 다음 배열은 입력값과 출력값으로 다음처럼 사용
# wor -> X, d -> Y
# woo -> X, d -> Y
seq_data = ['word', 'wood', 'deep', 'dive', 'cold', 
            'cool', 'load', 'love', 'kiss', 'kind']

def make_batch(seq_data):
    input_batch = []
    target_batch = []
    
    for seq in seq_data:
        # 여기서 생성하는 input_batch 와 target_batch 는
        # 알파벳 배열의 인덱스 번호이다.
        # [22, 14, 17] [22, 14, 14] [3, 4, 4] [3, 8, 21] ...
        input_ = [num_dic[n] for n in seq[:-1]]
        target = num_dic[seq[-1]]
        # one-hot 인코딩
        input_batch.append(np.eye(dic_len)[input_])
        target_batch.append(target)
    
    return input_batch, target_batch

In [14]:
tf.reset_default_graph()
###################
# Hyper-Parameter #
###################
learning_rate = 0.01
n_hidden = 128
total_epoch = 30
# time step: [1 2 3] => 3
# RNN을 구성하는 시퀀스의 개수
n_step = 3
# 입력값 크기. 알파벳에 대한 one-hot 인코딩이므로 26개가 된다.
# 예) c => [0 0 1 0 0 0 0 0 0 0 0 ... 0]
# 출력값도 입력값과 마찬가지로 26개의 알파벳으로 분류
n_input = n_class = dic_len

###############
# RNN 모델 구성 #
###############
inputs = tf.placeholder(tf.float32, [None, n_step, n_input])
# loss function에 sparse_softmax_cross_entropy_with_logits를 사용하므로
# label을 one-hot 인코딩이 아니라 그대로 사용한다.
labels = tf.placeholder(tf.int32, [None])

weights = tf.Variable(tf.truncated_normal([n_hidden, n_class], stddev=0.01))
bias = tf.Variable(tf.truncated_normal([n_class], stddev=0.1))

# LSTM Cell-1 생성
cell1 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
# Overfitting 방지를 위한 Dropout
cell1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob=0.5)
# LSTM Cell-2 생성
cell2 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)

# cell1 & cell2 셀을 조합한 LSTM Cell을 생성한다.
multi_cell = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2])

# tf.nn.dynamic_rnn 함수를 이용해 LSTM Network를 만듦
outputs, states = tf.nn.dynamic_rnn(multi_cell, inputs, dtype=tf.float32)
outputs = outputs[:, -1, :]
logits = tf.matmul(outputs, weights) + bias

# loss & optimizer
cross_entropy = tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, 
                                                       labels=labels))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

# accuracy
prediction = tf.cast(tf.argmax(logits, 1), tf.int32)
correct_mask = tf.equal(prediction, labels)
accuracy = tf.reduce_mean(tf.cast(correct_mask, tf.float32))

In [16]:
#################
# Training LSTM #
#################
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    batch_x, batch_y = make_batch(seq_data)
    
    for epoch in range(1, total_epoch+1):
        loss, _ = sess.run([cross_entropy, optimizer], 
                           feed_dict={inputs: batch_x, labels: batch_y})
        
        print('Epoch: {:04d}'.format(epoch), 
              'cost = {:.6f}'.format(loss))
        
    print('최적화 완료')
    
    predict, acc = sess.run([prediction, accuracy], 
                            feed_dict={inputs: batch_x, labels: batch_y})

print('Tes Acc = {:.6f}'.format(acc))

Epoch: 0001 cost = 3.324870
Epoch: 0002 cost = 3.294429
Epoch: 0003 cost = 3.202497
Epoch: 0004 cost = 2.974684
Epoch: 0005 cost = 2.430999
Epoch: 0006 cost = 1.685881
Epoch: 0007 cost = 1.356830
Epoch: 0008 cost = 1.488095
Epoch: 0009 cost = 1.428164
Epoch: 0010 cost = 1.339205
Epoch: 0011 cost = 1.381152
Epoch: 0012 cost = 1.383293
Epoch: 0013 cost = 1.321613
Epoch: 0014 cost = 1.278969
Epoch: 0015 cost = 1.256775
Epoch: 0016 cost = 1.220558
Epoch: 0017 cost = 1.186458
Epoch: 0018 cost = 1.114553
Epoch: 0019 cost = 1.022924
Epoch: 0020 cost = 1.056550
Epoch: 0021 cost = 0.994388
Epoch: 0022 cost = 0.892078
Epoch: 0023 cost = 0.934900
Epoch: 0024 cost = 0.830486
Epoch: 0025 cost = 0.851086
Epoch: 0026 cost = 0.757711
Epoch: 0027 cost = 0.721731
Epoch: 0028 cost = 0.702509
Epoch: 0029 cost = 0.598586
Epoch: 0030 cost = 0.639712
최적화 완료
Tes Acc = 0.700000


In [19]:
predict_words = []
for idx, val in enumerate(seq_data):
    last_char = char_arr[predict[idx]]
    predict_words.append(val[:3] + last_char)
    
print('\n=== 예측 결과 ===')
print('입력값:', [w[:3] + ' ' for w in seq_data])
print('예측값:', predict_words)
print('정확도:', acc)


=== 예측 결과 ===
입력값: ['wor ', 'woo ', 'dee ', 'div ', 'col ', 'coo ', 'loa ', 'lov ', 'kis ', 'kin ']
예측값: ['word', 'wood', 'deep', 'dive', 'cold', 'cood', 'load', 'lovd', 'kisd', 'kind']
정확도: 0.7
